In [234]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [235]:
customer_df = pd.read_csv('olist_customers_dataset.csv')
customer_df.shape
customer_df['customer_unique_id'].nunique()

#view of original cust_df

96096

In [236]:
orders_df = pd.read_csv('olist_orders_dataset.csv')
orders_df.head(50)
orders_df.shape

#view of original orders_df

(99441, 8)

In [237]:
orderitems_df = pd.read_csv('olist_order_items_dataset.csv')
orderitems_df.shape
orderitems_df['order_id'].nunique()

#view of original order_items_df - 112650 is fine

98666

In [238]:
products_df = pd.read_csv('olist_products_dataset.csv')
products_df.head(5)
products_df['product_id'].nunique()

#og products df

32951

In [239]:
product_transl_df = pd.read_csv('product_category_name_translation.csv')
product_transl_df.head(5)
product_transl_df.shape


(71, 2)

In [240]:
cust_orders_df = pd.merge(customer_df, orders_df, on = 'customer_id')
cust_orders_df.shape

cust_orders_df['customer_unique_id'].nunique()

#view of original custumer orders df

96096

In [241]:
orders_and_items = pd.merge(orders_df, orderitems_df, on='order_id')
orders_and_items.head(5)
orders_and_items.shape

#view of orders + items in that order

(112650, 13)

In [242]:
prod_order_items = pd.merge(orders_and_items, products_df, on = 'product_id', how='right')
prod_order_items.head(5)

prod_order_items.shape

# view of orders + items in that order + what each item is

(112650, 21)

In [243]:
prod_order_items = pd.merge(prod_order_items, product_transl_df, on='product_category_name', how='left')
prod_order_items.drop(['product_category_name'], axis=1, inplace=True)
prod_order_items.head(50)

prod_order_items.shape
# same as above except adding product translation + dropping portuguese description

(112650, 21)

In [244]:
final_df = pd.merge(prod_order_items, customer_df, on = 'customer_id', how='right')
final_df.head(50)


final_df['customer_unique_id'].nunique()
#adding customer_unique_id

96096

In [245]:
cols = final_df.columns.tolist()
cols

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'order_item_id',
 'product_id',
 'seller_id',
 'price',
 'freight_value',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm',
 'product_category_name_english',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']

In [246]:
cols = ['order_id',
 'customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'order_item_id',
 'product_id',
 'product_category_name_english',
 'seller_id',
 'price',
 'freight_value',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']

In [247]:
final_df = final_df[cols]

In [248]:
final_df.head(5)
final_df.shape
final_df['customer_unique_id'].nunique()


96096

In [249]:
test = pd.DataFrame(final_df.groupby(['customer_unique_id'])['order_id'].count())
test.shape

# final_df['customer_unique_id'].nunique() -- num of unique doesnt match Kaggle but i think Kaggle wrong

(96096, 1)

In [250]:
test.columns = ['num_orders']

In [251]:
more_than_one = test.query('num_orders > 1').shape

In [252]:
'Out of {} customers in this dataset, {} customers had more than 1 order - which is {}% of all customers.'.format(test.shape[0], more_than_one[0], round((more_than_one[0]/test.shape[0])*100,2))

'Out of 96096 customers in this dataset, 11869 customers had more than 1 order - which is 12.35% of all customers.'

In [253]:
final_df.dtypes

order_id                          object
customer_id                       object
customer_unique_id                object
customer_zip_code_prefix           int64
customer_city                     object
customer_state                    object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
order_item_id                    float64
product_id                        object
product_category_name_english     object
seller_id                         object
price                            float64
freight_value                    float64
product_name_lenght              float64
product_description_lenght       float64
product_photos_qty               float64
product_weight_g                 float64
product_length_cm                float64
product_height_cm                float64
product_width_cm

In [254]:
final_df['order_purchase_timestamp'] = pd.to_datetime(final_df['order_purchase_timestamp'])

In [255]:
print('Data is from {} to {}.'.format(str(final_df['order_purchase_timestamp'].min()), str(final_df['order_purchase_timestamp'].max())))


Data is from 2016-09-04 21:15:19 to 2018-09-03 09:06:57.


In [281]:
#Average num of items in each order

tt = pd.DataFrame(final_df.groupby(['order_id'])['order_id'].count())
tt['order_id'].mean()


#amongst all customers, people tend to order around 1 item per order

(98666, 1)

In [280]:
test.query('num_orders > 1').mean()

#amongst those customers that order more than 1 item, they usually only do around 2-3

num_orders    2.451681
dtype: float64